**Задача 1**

На основе алгоритма DeepSORT реализуйте возможность подсчета количества объектов и какого класса пересекает заданную линию в обеих направлениях.

**Задача 2**

На основе алгоритма DeepSORT реализуйте сохранение метаданных трекинга в файл (id, координаты, класс)

In [ ]:
!pip install ikomia >> None

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
import requests
import cv2
from ikomia.dataprocess.workflow import Workflow
import csv

In [ ]:
video_path = "pedestrian.mp4"
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    exit()
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
duration = frame_count / fps

wf = Workflow()
wf.add_task(name="infer_yolo_v7", auto_connect=True)
tracking = wf.add_task(name="infer_deepsort", auto_connect=True)
tracking.set_parameters({
    "categories": "all",
    "conf_thres": "0.5",
    "max_age": "50",
    "min_hits": "3",
    "iou_threshold": "0.2",
    "cosine_threshold": "0.2",
    "nn_budget": "100",
    "use_cuda": "True"
})

stream = cv2.VideoCapture(video_path)
frame_width = int(stream.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(stream.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = stream.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('cross_count_output.mp4', fourcc, frame_rate, (frame_width, frame_height))

line_x = frame_width // 2
counts = {"right": {}, "left": {}}
prev_centroids = {}

while True:
    ret, frame = stream.read()
    if not ret:
        break
    wf.run_on(array=frame)
    image_out = tracking.get_output(0)
    obj_detect = tracking.get_output(1)
    vis = image_out.get_image_with_graphics(obj_detect)
    frame_vis = cv2.cvtColor(vis, cv2.COLOR_RGB2BGR)
    detections = obj_detect.get_objects()
    current_centroids = {}
    for det in detections:
        tid = det.id
        cls = det.label
        x, y, w, h = det.box
        cx = int(x + w/2)
        cy = int(y + h/2)
        current_centroids[tid] = (cx, cy, cls)
        if tid in prev_centroids:
            px, _ = prev_centroids[tid][:2]
            if px < line_x and cx >= line_x:
                counts["right"].setdefault(cls, 0)
                counts["right"][cls] += 1
            elif px > line_x and cx <= line_x:
                counts["left"].setdefault(cls, 0)
                counts["left"][cls] += 1
    prev_centroids = current_centroids
    cv2.line(frame_vis, (line_x, 0), (line_x, frame_height), (0, 0, 255), 2)
    for direction, y_pos in [("right", 30), ("left", 60)]:
        text = ' '.join(f"{k}:{v}" for k, v in counts[direction].items())
        cv2.putText(frame_vis, f"{direction} {text}", (10, y_pos), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    out.write(frame_vis)

stream.release()
out.release()

Will run on cuda
Fusing layers... 
Successfully loaded imagenet pretrained weights from "/root/Ikomia/Plugins/Python/infer_deepsort/models/checkpoints/osnet_x1_0_imagenet.pth"
Workflow Untitled run successfully in 660.269487 ms.
Workflow Untitled run successfully in 145.38858 ms.
Workflow Untitled run successfully in 139.071865 ms.
Workflow Untitled run successfully in 161.002861 ms.
Workflow Untitled run successfully in 149.46021 ms.
Workflow Untitled run successfully in 146.925477 ms.
Workflow Untitled run successfully in 143.168839 ms.
Workflow Untitled run successfully in 144.71515 ms.
Workflow Untitled run successfully in 187.534867 ms.
Workflow Untitled run successfully in 137.705675 ms.
Workflow Untitled run successfully in 137.747776 ms.
Workflow Untitled run successfully in 143.890393 ms.
Workflow Untitled run successfully in 146.483367 ms.
Workflow Untitled run successfully in 142.875203 ms.
Workflow Untitled run successfully in 139.38670100000002 ms.
Workflow Untitled run su

In [ ]:
stream = cv2.VideoCapture(video_path)
writer = csv.writer(open('tracking_metadata.csv', 'w', newline=''))
writer.writerow(['frame', 'track_id', 'xmin', 'ymin', 'xmax', 'ymax', 'class'])

frame_idx = 0
while True:
    ret, frame = stream.read()
    if not ret:
        break
    wf.run_on(array=frame)
    obj_detect = tracking.get_output(1)
    detections = obj_detect.get_objects()
    for det in detections:
        tid = det.id
        cls = det.label
        x, y, w, h = det.box
        x1, y1 = x, y
        x2, y2 = x + w, y + h
        writer.writerow([frame_idx, tid, x1, y1, x2, y2, cls])
    frame_idx += 1
stream.release()

Workflow Untitled run successfully in 196.57315599999998 ms.
Workflow Untitled run successfully in 240.99003299999998 ms.
Workflow Untitled run successfully in 239.27193599999998 ms.
Workflow Untitled run successfully in 340.388978 ms.
Workflow Untitled run successfully in 167.759231 ms.
Workflow Untitled run successfully in 361.88854000000003 ms.
Workflow Untitled run successfully in 280.990573 ms.
Workflow Untitled run successfully in 252.461244 ms.
Workflow Untitled run successfully in 169.904427 ms.
Workflow Untitled run successfully in 354.405729 ms.
Workflow Untitled run successfully in 288.622049 ms.
Workflow Untitled run successfully in 345.327188 ms.
Workflow Untitled run successfully in 248.95498199999997 ms.
Workflow Untitled run successfully in 557.760005 ms.
Workflow Untitled run successfully in 267.851504 ms.
Workflow Untitled run successfully in 365.83053199999995 ms.
Workflow Untitled run successfully in 382.439834 ms.
Workflow Untitled run successfully in 167.080742 ms

**Задача 3**

Выведите время жизни каждого трека. То есть отслеживаете отдельный id и считаете его время жизни.

In [ ]:
import pandas as pd
import cv2

In [ ]:
video_path = 'cross_count_output.mp4'
video = cv2.VideoCapture(video_path)

fps = video.get(cv2.CAP_PROP_FPS)
video.release()

fps

25.0

In [ ]:
df = pd.read_csv('tracking_metadata.csv')
df

,frame,track_id,xmin,ymin,xmax,ymax,class
0,0,1,410.0,305.0,490.0,370.0,car
1,0,2,820.0,292.0,1132.0,563.0,car
2,0,3,99.0,311.0,247.0,428.0,car
3,0,4,900.0,267.0,1263.0,715.0,car
4,0,5,490.0,303.0,549.0,354.0,car
...,...,...,...,...,...,...,...
1813,147,146,176.0,282.0,268.0,355.0,car
1814,147,152,433.0,291.0,498.0,360.0,car
1815,147,153,645.0,285.0,675.0,324.0,car
1816,147,155,1063.0,254.0,1096.0,287.0,person


In [18]:
res = []
for id in df['track_id'].unique():
    res.append((int(id), (len(df[df['track_id'] == id]) / fps)))
pd.DataFrame(res, columns=['track', 'Время жизни, сек']).head()

,track,"Время жизни, сек"
0,1,2.08
1,2,5.92
2,3,1.84
3,4,5.92
4,5,1.24
